## Imports

In [ ]:
import numpy as np
import tiktoken  # Tokenizer-Bibliothek für GPT-2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 1. Tokenisierung:
- Tokenisierung ist der Prozess der Umwandlung von Text in kleinere Einheiten (Tokens).
- Tokens können Zeichen ("a", "b"), Wörter ("Hallo", "Baum") oder Subwörter ("Ha", "Ba") sein.
- LLMs arbeiten mit Token-Sequenzen anstelle von Rohtext → Tokenisierung ist daher das Vorbereiten des Textes für die Eingabe in das LLM :)




### 1.1 Datensatz erstellen:

In [ ]:
class DatasetForGPT(Dataset):
    def __init__(self, max_length, stride, tokenizer, txt):
        """
        Erstellt ein Dataset aus einem Text für ein GPT-basiertes Modell.

        Parameter:
        - txt: Der Eingabetext als String. (z.B. ein Opensource Buch der Seite Projekt Gutenberg https://www.projekt-gutenberg.org/)
        - tokenizer: Der GPT2-Tokenizer zur Tokenisierung des Textes.
        - max_length: Maximale Länge einer Token-Sequenz.
        - stride: Schrittweite für die Erstellung überlappender Sequenzen.
        """
        self.input_ids = []  # Liste zur Speicherung der Eingabe-Tokens
        self.target_ids = []  # Liste zur Speicherung der Ziel-Tokens (verschobene Sequenz)

        # Den gesamten Text in Tokens umwandeln
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Erzeuge überlappende Sequenzen aus den Token-IDs (siehe Erklärung 1)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]  # Eingabesequenz
            target_chunk = token_ids[
                           i + 1: i + max_length + 1]  # Zielsequenz (verschoben um 1 Token -> siehe Bild 1: Erklärung 1 )

            # Speichern der Tensor-Repräsentation der Sequenzen
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        """ Gibt die Anzahl der Trainingsbeispiele zurück."""
        return len(self.input_ids)

    def __getitem__(self, idx):
        """ Gibt ein Trainingsbeispiel bestehend aus (Eingabe, Ziel) zurück."""
        return self.input_ids[idx], self.target_ids[idx]


##### Erklärung 1
Das bedeutet:
- Das Modell bekommt eine Eingabesequenz (input_chunk), z. B. "LLMs learn to predict one"
- Die Zielsequenz (target_chunk) ist dann dieselbe Sequenz, aber um ein Token nach rechts verschoben, sodass das Modell lernen soll, das nächste Token vorherzusagen.

→ Genau wie im Bild (Quelle: Raschka 2025)
###### Bild 1 Sliding Window Approach
![Image 1 Sliding Window Approach](images/Image1_sliding_window_approach.png)

Das Modell sieht nur den bisherigen Kontext (blau markiert).
Das Modell soll das nächste Wort (rot markiert) vorhersagen.
Es kann zukünftige Wörter nicht direkt sehen, sondern muss sie aus den bisherigen Token ableiten. Während des Trainings bekommt das Modell eine Sequenz (input_chunk) als Eingabe und versucht, das nächste Token (target_chunk) vorherzusagen. Durch viele Wiederholungen lernt das Modell dann grammatische Strukturen, Satzbedeutungen und sogar komplexe Zusammenhänge.

### 1.2 Dataloader erzeugen:

Der DataLoader hilft, die Daten effizient für das LLM bereitzustelle.


In [ ]:
def create_dataloader(txt, batch_size=4, max_length=256,
                      stride=128, shuffle=True, drop_last=True, num_workers=0):
    """
    Erstellt einen DataLoader für das Training eines LLMs.

    Parameter:
    - txt: Eingabetext als String.
    - batch_size: Anzahl der Samples pro Batch.
    - max_length: Maximale Token-Sequenzlänge.
    - stride: Schrittweite für die Erzeugung überlappender Sequenzen.
    - shuffle: Ob die Reihenfolge der Sequenzen zufällig gemischt wird.
    - drop_last: Ob das letzte Batch verworfen wird, falls es unvollständig ist.
    - num_workers: Anzahl der Threads für die Datenverarbeitung.

    Rückgabe:
    - Ein DataLoader-Objekt für das Training.
    """
    # Initialisiere den GPT-2 Tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Erstelle das Dataset
    dataset = DatasetForGPT(max_length, stride, tokenizer, txt)

    # Erstelle den DataLoader aud der torch Lib (Erklärung 2):
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
    #Shuffle sorgt für zufällige Reihenfolge -> Damit das Modell nicht immer dieselbe Reihenfolge der Daten sieht (wichtig für das Training).
    # Drop_last vermeidet ungleich große Batches
    # Falls die Anzahl der Daten nicht genau durch batch_size teilbar ist, werden unvollständige Batches verworfen.

    return dataloader


##### Erklärung 2:

Ein DataLoader bereitet die Daten für das Modelltraining vor. In diesem Fall fasst der Data Loader mehrere Sequenzen (des Sliding Window Approaches) zu Batches zusammen:

Das Modell verarbeitet nicht einen einzelnen Satz nach dem anderen, sondern mehrere Sequenzen gleichzeitig (z. B. 4 auf einmal, wenn batch_size=4 ist).
###### Beispiel:
**Batch 1:**
- Eingabe:  `["LLMs learn to predict", "learn to predict one", "to predict one word", "predict one word at"]`
- Ziel:     `["learn to predict one", "to predict one word", "predict one word at", "one word at a"]`

**Batch 2:**
- Eingabe:  `["one word at a", "word at a time", "at a time <|endoftext|>", "..."]`
- Ziel:     `["word at a time", "at a time <|endoftext|>", "...", "..."]`




### 1.3 Code testen

In [5]:
with open("test_text", "r") as f:
    test_text = f.read()

# DataLoader erstellen
dataloader = create_dataloader(test_text, batch_size=2, max_length=6, stride=3)

# Ersten Batch ausgeben
for batch in dataloader:
    inputs, targets = batch
    print("\n=== Erster Batch ===")
    print("Input Shape:", inputs.shape)  # Erwartet: (batch_size, max_length)
    print("Target Shape:", targets.shape)  # Erwartet: (batch_size, max_length)
    print("\nEingabe Batch:", inputs)
    print("Ziel Batch:", targets)
    break  # Nur den ersten Batch ausgeben




=== Erster Batch ===
Input Shape: torch.Size([2, 6])
Target Shape: torch.Size([2, 6])

Eingabe Batch: tensor([[ 3436,   351,   683,    13,   314,   550],
        [  835,   286,  1762,    30,  2011, 29483]])
Ziel Batch: tensor([[  351,   683,    13,   314,   550,  1908],
        [  286,  1762,    30,  2011, 29483,  2540]])


#### 1.4 Zusammenfassung Tokenisierung
- Das DatasetForGPT erzeugt nur die überlappenden Sequenzen (Sliding Window Approach, siehe Bild 1) und speichert bereits alle Input- und Target Sequenzen
- Der DataLoader fasst diese dann in Batches zusammen, die dann im Training verwendet werden können.

---

# 2. Attention

### 2.1 Warum brauchen wir Attention?
Vor der Einführung von **Self-Attention** wurden **Recurrent Neural Networks (RNNs)** und **Encoder-Decoder-Architekturen** für die Verarbeitung von Sequenzen verwendet. Allerdings hatten sie zwei große Probleme:

- **Langstreckenabhängigkeiten** wurden nicht gut erfasst, da Informationen aus weit entfernten Tokens schwer erhalten blieben.
- **Feste Reihenfolge & Verlust von Kontext** – Die Modelle konnten frühere Tokens nicht direkt abrufen, sondern mussten sich auf eine latente **Hidden State-Repräsentation** verlassen.

**Lösung:**
**Self-Attention**, ein Mechanismus, der es jedem Token erlaubt, mit allen anderen Tokens in der Sequenz in Beziehung zu treten und zu bestimmen, welche Tokens wichtig sind.


### 2.2 Was ist Self-Attention?
Self-Attention ist das Herzstück moderner LLMs wie GPT.
Es ermöglicht dem Modell, zu jedem Zeitpunkt zu bestimmen, welche Wörter in einem Satz für ein bestimmtes Token am wichtigsten sind. Die „Self“-Komponente in Self-Attention bedeutet, dass das Modell die Aufmerksamkeitsgewichte innerhalb einer einzelnen Eingabesequenz berechnet. Es analysiert die Beziehungen und Abhängigkeiten zwischen verschiedenen Positionen innerhalb der Sequenz selbst – beispielsweise zwischen Wörtern in einem Satz oder Pixeln in einem Bild. Dadurch kann das Modell relevante Informationen aus dem gesamten Kontext ziehen, anstatt sich nur auf benachbarte Elemente zu verlassen.

![Image1_Attention_Mechanism](images/Image2_Attention_Mechanisms.png)


#### **2.3 Berechnung der Attention-Scores**

![Image1_Attention_Mechanism](images/Image3_3stepts.png)



#### **2.3.1 Erzeuge Queries, Keys und Values für jedes Token**

![Image4](images/Image4_first_step_attention.png)


### **Berechnung von Query, Key und Value in Self-Attention**

Im ersten Schritt des **Self-Attention-Mechanismus** mit **trainierbaren Gewichtsmatrizen** werden die **Query $(q)$ **, **Key $(k)$** und **Value $(v)$**-Vektoren für die Eingabeelemente $(x)$ berechnet.

Ähnlich wie in den vorherigen Abschnitten wird das **zweite Eingabeelement** $(x(2))$ als **Query-Input** ausgewählt. Der Query-Vektor $(q(2))$ wird durch eine **Matrixmultiplikation** zwischen $(x(2))$ und der **Gewichtsmatrix** $(W_Q)$ berechnet:

$
q(2) = x(2) \cdot W_Q
$

Ebenso werden die **Key- und Value-Vektoren** durch Matrixmultiplikation mit den jeweiligen **Gewichtsmatrizen** $(W_K)$ und $(W_V)$ erzeugt:

$
k(2) = x(2) \cdot W_K, \quad v(2) = x(2) \cdot W_V
$

#### **2.3.2. Berechnung der Similarity (Attention Scores)**

![Image 5](images/Image5_Step2_attention.png)


Die Attention Scores ergeben sich aus dem Skalarprodukt zwischen den transformierten Query- und Key-Vektoren.
Anstatt einzelne Scores für jedes Token zu berechnen, verwenden wir Matrixmultiplikation, um die Attention Scores für alle Sequenzpositionen gleichzeitig zu berechnen:


Die Relevanz eines Tokens für ein anderes wird mit einem **Skalarprodukt** zwischen $(Q)$ und $(K^T)$ berechnet:

$\text{Scores} = Q K^T$

Je größer das Ergebnis, desto relevanter ist das Token für den aktuellen Kontext.
Da die Dot-Produkte sehr große Werte annehmen können, normalisieren wir sie, indem wir sie durch die Quadratwurzel der Key-Dimension dk teilen (Softmax Funktion).#
$
\alpha = \text{Softmax} \left(\frac{Q K^T}{\sqrt{d_k}} \right)
$

---
(Skalarproduct:
$
a \cdot b = \sum_{i=1}^{n} a_i b_i
$)

Beispiel für das Skalarprodukt

$
a = \begin{bmatrix} 2 \\ 3 \\ 4 \end{bmatrix}, \quad
b = \begin{bmatrix} 1 \\ 0 \\ -1 \end{bmatrix}
$

Das Skalarprodukt wird berechnet als:

$
a \cdot b = (2 \cdot 1) + (3 \cdot 0) + (4 \cdot (-1))
$

$
a \cdot b = 2 + 0 - 4 = -2
$

Ergebnis: Das Skalarprodukt von $(a)$ und $(b)$** ist $(-2)$.
---



#### **2.3.3. Softmax zur Normalisierung der Gewichte**
![Image 6](images/Image6_step3_attention.png)

Softmax zur Normalisierung der Scores in Wahrscheinlichkeiten
Nachdem die Attention-Scores $(\omega)$ berechnet wurden, werden diese mithilfe der Softmax-Funktion normalisiert,
um die Attention-Gewichte $(\alpha)$ zu erhalten.
Dabei wird sichergestellt, dass alle Attention-Gewichte positiv sind und ihre Summe 1 ergibt.

$\alpha = \text{Softmax} \left(\frac{QK^T}{\sqrt{d_k}} \right)$


#### **4. Erzeuge Kontextvektoren als gewichtete Summe der Values**
Schließlich werden die normalisierten Scores genutzt, um die **Values** zu gewichten:

$Z = \alpha V$




### 2.4 Causal Attention für LLMs
GPT-Modelle nutzen eine spezielle Form der Self-Attention, **Causal Attention**, um sicherzustellen, dass ein Modell bei der **Textgenerierung nur auf frühere Tokens schaut**.

**Wie wird das erreicht?**
- Eine **Causal Mask** wird auf die Attention Scores angewendet, um **zukünftige Tokens auszublenden**.
- Mathematisch bedeutet das, dass Werte oberhalb der Hauptdiagonale der Attention-Matrix mit **\(-\infty\)** belegt werden.



### 2.5 Multi-Head Attention – Erweiterung von Self-Attention
Anstatt nur **einen einzigen Self-Attention-Mechanismus** zu nutzen, **wird dieser in mehrere parallele "Köpfe" (Heads) aufgeteilt**.

### **Warum Multi-Head Attention?**
**Lernt verschiedene Aspekte einer Sequenz gleichzeitig**
- Jeder Kopf fokussiert sich auf **unterschiedliche Muster** (Syntax, Semantik, Langstreckenabhängigkeiten).

**Verbessert die Stabilität des Trainings**
- Mehrfache unabhängige Berechnungen gleichen mögliche Fehler einzelner Heads aus.

**Erhöht die Repräsentationskraft des Modells**
- Durch die Kombination mehrerer Perspektiven erhält das Modell **tiefere Einsichten** in die Bedeutung eines Satzes.



### **Technische Umsetzung**
- **Jeder Kopf hat eigene Gewichtsmatrizen** $(W_Q, W_K, W_V)$
- **Die Self-Attention wird parallel für jeden Kopf berechnet.**
- Danach werden die Ergebnisse aller Heads **konkateniert** und durch eine weitere Projektion zusammengeführt:

$Z_{\text{final}} = W_O [Z_1, Z_2, ..., Z_h]$

wobei $(W_O)$ eine weitere Gewichtsmatrix ist, die das gesamte Ergebnis transformiert.



### 2.6 Unterschiedliche Implementierungen von Multi-Head Attention
Rashka (2024) beschreibt zwei Implementierungsansätze:

#### **Einfache Multi-Head Attention durch Stacken von mehreren Attention-Modulen**
Mehrere **unabhängige Self-Attention-Blöcke** werden nebeneinander gelegt und deren Ergebnisse kombiniert.
**Nachteil**: Rechenaufwändig, da jede Attention **separat** berechnet wird.

#### **Effiziente Multi-Head Attention mit paralleler Berechnung**
**Ein einziger linearer Layer** erzeugt **Queries, Keys und Values** für alle Heads gleichzeitig.
Diese werden dann **in verschiedene Heads gesplittet**, sodass die Berechnungen **parallel stattfinden können**.
**Vorteil**: Spart **Speicher** und ist **rechenoptimiert**.


#### Dropout in Attention – Reduzierung von Overfitting
Zusätzlich zu **Causal Masks** wird **Dropout auf die Attention Scores angewendet**, um:
- **Overfitting zu vermeiden**, indem zufällige Gewichte auf 0 gesetzt werden.
- **Das Modell zu zwingen, robustere Muster zu lernen.**

In [ ]:
import torch
import torch.nn as nn


class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        """
        Implementiert Multi-Head Attention, ein zentraler Mechanismus in Transformer-Modellen.

        Parameter:
        - d_in: Eingabe-Dimension (z. B. Anzahl der Merkmale pro Token).
        - d_out: Ausgabe-Dimension (muss durch num_heads teilbar sein).
        - context_length: Maximale Sequenzlänge.
        - dropout: Dropout-Wahrscheinlichkeit zur Regularisierung.
        - num_heads: Anzahl der parallelen Attention-Heads.
        - qkv_bias: Ob die linearen Projektionsmatrizen für Q, K und V einen Bias enthalten.
        """
        super().__init__()
        assert d_out % num_heads == 0, "d_out muss durch num_heads teilbar sein"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Dimension jedes Attention-Heads

        # Lineare Transformationen für Query, Key und Value
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # Finale lineare Projektion nach der Attention-Berechnung
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)

        # Maskierung für zukünftige Tokens (Causal Mask für autoregressive Modelle)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        """
        Führt den Multi-Head Attention Mechanismus aus.
        - x: Eingabe-Tensor der Form (Batch, Sequenzlänge, d_in)
        """
        b, num_tokens, d_in = x.shape  # Batch-Größe, Sequenzlänge, Eingabe-Dimension

        # Berechnung von Query, Key und Value
        keys = self.W_key(x)  # (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # Aufteilen in mehrere Köpfe (num_heads)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1,
                                                                                 2)  # (b, num_heads, num_tokens, head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)

        # Berechnung der Attention Scores mittels Skaliertem Dot-Product Attention
        attn_scores = queries @ keys.transpose(2, 3)  # (b, num_heads, num_tokens, num_tokens)
        attn_scores = attn_scores / (self.head_dim ** 0.5)  # Skalierung für stabilere Gradienten

        # Anwenden der Maskierung, um zukünftige Tokens auszublenden (Causal Masking)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        # Softmax zur Normalisierung der Scores in Wahrscheinlichkeiten
        attn_weights = torch.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Berechnung der gewichteten Summen der Values
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Rekombinieren der Heads zur ursprünglichen Ausgabe-Dimension
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # Finale lineare Transformation

        return context_vec


---

#### Sources:

Raschka, Sebastian (2025): Build a Large Language Model (from scratch). Shelter Island: Manning (From scratch series). Online verfügbar unter https://ebookcentral.proquest.com/lib/kxp/detail.action?docID=31657639.